In [1]:
import os
from urllib.request import urlopen
from urllib.request import urlretrieve
from urllib.parse import quote
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from operator import eq

In [2]:
def getBs(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs = BeautifulSoup(html, 'html.parser')
    except AttributeError as e:
        return None
    return bs



In [3]:
## 크롤링 해야될 목록들
mainUrl = 'http://modernsweet.co.kr'

def get_category(url):
    bs = getBs(url)
    category = bs.ul.findAll('li',{'class':'xans-record-'})
    
    for item in category:
        print(item.a.get_text(), ' : ',item.a.attrs['href'])
    return category

get_category(mainUrl)

BEST 50  :  /product/list.html?cate_no=76
ALL  :  /product/list.html?cate_no=70
OUTER  :  /product/list.html?cate_no=24
TOP  :  /product/list.html?cate_no=26
SHIRT  :  /product/list.html?cate_no=27
PANTS  :  /product/list.html?cate_no=28
BAG/SHOES  :  /product/list.html?cate_no=42
ACC  :  /product/list.html?cate_no=43
PERSONAL ORDER  :  /product/list.html?cate_no=44


[<li class="xans-record-"><a href="/product/list.html?cate_no=76">BEST 50</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=70">ALL</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=24">OUTER</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=26">TOP</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=27">SHIRT</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=28">PANTS</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=42">BAG/SHOES</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=43">ACC</a></li>,
 <li class="xans-record-"><a href="/product/list.html?cate_no=44">PERSONAL ORDER</a></li>]

In [4]:
## 다음 페이지 알아내기

def get_nextpage(url):
    bs = getBs(url)
    next_page = bs.find('img',{'alt':'다음 페이지'})
    next_page = next_page.parent.attrs['href']
    
    if eq(next_page,'#none'):
        return next_page
    
    andp = next_page.find('&') # 구분자 찾기
    next_page= next_page[andp:]
    return next_page

## 범위 벗어나면 #none
get_nextpage('http://modernsweet.co.kr/product/list.html?cate_no=70&page=5')

'&page=6'

In [5]:
## 페이지 내부에서 옷 목록 가져오기

def get_item(url):
    bs = getBs(url)
    item_list = bs.findAll('div',{'class':'box'})
    count = 1
    for item in item_list:
        try:
            print(count,') name : ',item.img.attrs['alt'])
            print('url :',item.a.attrs['href'])
            count+=1
        except AttributeError as e:
            print('no item')
    return item_list
item_list =  get_item('http://modernsweet.co.kr/product/list.html?cate_no=70')

1 ) name :  아리조나 반팔 티 4color
url : /product/detail.html?product_no=1184&cate_no=70&display_group=1
2 ) name :  아레스 박스 오버 반팔티 3color
url : /product/detail.html?product_no=1183&cate_no=70&display_group=1
3 ) name :  SAROLEA 박스 나시티 3color
url : /product/detail.html?product_no=1182&cate_no=70&display_group=1
4 ) name :  타임 절개 카라 반팔티 8color
url : /product/detail.html?product_no=1181&cate_no=70&display_group=1
5 ) name :  코튼 워싱 카고 반바지 4color
url : /product/detail.html?product_no=1180&cate_no=70&display_group=1
6 ) name :  샌더스 반팔 니트 5color
url : /product/detail.html?product_no=1179&cate_no=70&display_group=1
7 ) name :  바이오워싱 코튼 반바지 3color
url : /product/detail.html?product_no=1178&cate_no=70&display_group=1
8 ) name :  네이쳐 린넨 긴팔 티 7color
url : /product/detail.html?product_no=1177&cate_no=70&display_group=1
9 ) name :  컴포트 스판 10부 슬랙스 4color
url : /product/detail.html?product_no=1176&cate_no=70&display_group=1
10 ) name :  코튼 스트라이프 반팔 티 3color
url : /product/detail.html?product_no=1175&cate_no

In [6]:
### 현제 페이지에서 이미지 모두 가져오기
downloadDirectory = 'download'

def getDownloadLink(source):
    mainUrl = 'http://modernsweet.co.kr'
    source = quote(source,safe=':/')
    return mainUrl+source

def get_img(url,filename):
    bs = getBs(url)
    downpath = 'downdown/'+filename+'/'
    item_information = bs.find('div',{'id':'prdDetail'})
    img_list = item_information.findAll('img')
    count = 1
    for img in img_list:
        print(getDownloadLink(img.attrs['ec-data-src']))
        urlretrieve(getDownloadLink(img.attrs['ec-data-src']),downpath+str(count)+'.jpg')
        count+=1


In [7]:
## 폴더 만들기
os.makedirs('downdown/hello')

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'downdown/hello'

In [8]:
def get_information(url):
    bs = getBs(url)
    info_list = bs.find('div',{'class':'cont'})
    info_list = info_list.findAll('span')
    count = 1
    text = ''
    for info in info_list:
        #print(count,':',info.get_text())
        #count+=1
        text += info.get_text()+'\n'
    return text

print(get_information('http://modernsweet.co.kr/product/detail.html?product_no=1180&cate_no=70&display_group=1'))


INFO
INFO
개인적으로도 강력 추천드리는 반바지 입니다:)
카고 형식으로 제작된 반바지에요.
탄탄한 면 소재이지만 답답하지 않고 짧은 기장감으로 제작되어
시원하고 캐쥬얼한 느낌으로 착용하기 좋으며
기본 무지 티셔츠와 매치하여도 
카고 디테일(포켓) 이 있어서 포인트를 주기 좋은 제품입니다:)
무엇보다 봄,가을 시즌 되면
맨투맨에 흰티 레이어드해서 입으면 정말 이쁜 제품이에요!
맨투맨에 흰티 레이어드해서 입으면 정말 이쁜 제품이에요!

무더운 여름에는 반팔티와 슬라브티 등을 매치하여
가볍고 편안하게 입으시고
FW 시즌되면 맨투맨과 티셔츠를 레이어드하여
남친룩을 연출하셔도 좋습니다:)
원단 자체가 워싱이 되어 제작된 제품이기 때문에
세탁 후에도 변형이나 뒤틀림이 전혀 없는 제품이라 매 시즌마다 꺼내어 입어주셔도 좋으며
허리 부분엔 뒷 밴딩으로 처리되어 있어 착용감도 굉장히 편하고
사이즈 또한 S,M,L,XL 까지 있어 남성분들 사이즈 선택에 폭도 넓습니다!
개인적으로는 컬러는 그레이, 블랙 제품이 어떤 상의와도 매치하기가 쉬워 추천드리며
남성분들 하나쯤 꼭 소장하시길 추천드리는 꿀템이라 생각합니다!
맨투맨과 연출된 코디컷은 곧 추가해서 보여드릴게요:)
COLOR
블랙, 오렌지, 화이트, 그레이
 
SIZE(cm)
S : 34(뒷밴딩) / 밑위 26 / 허벅지 32 / 밑단 28 / 총기장 39
M : 36(뒷밴딩) / 밑위 28 / 허벅지 33 / 밑단 29 / 총기장 40
L : 38(뒷밴딩) / 밑위 30 / 허벅지 34 / 밑단 30 / 총기장 41
XL : 40(뒷밴딩) / 밑위 32 / 허벅지 35 / 밑단 31 / 총기장 42
측정 위치에 따라 1-3CM 의 오차가 발생 할 수 있습니다.
FABRIC
면/스판
MODEL
174 CM / 63 KG
평소 28~29사이즈 착용 모델
S 사이즈 편안하게 착용하였습니다:)
DETAIL
비침 / 없음
두께감 / 보통
신축성 / 약간
그레이 / S 사이즈
DETAIL VIEW
화이트 / 그레이 / 오렌지 / 

In [9]:
def get_infofile(text,filename):

    path = 'downdown/'+filename+'/'
    f = open(path+filename+'.txt','w',-1, "utf-8")
    
    f.write(text)
    f.close()
    

In [10]:
def crawling():
    global mainUrl
    ## 메뉴 가져오기
    category = get_category(mainUrl)
    
    ##메뉴 크롤링 시작
    for page in category:
        print('start {} crawling'.format(page.a.get_text()))
        ## 타겟 페이지 설정
        target = mainUrl+page.a.attrs['href']
        count = 1
        while 1:
            print(count,' page')
            ##타겟 페이지 내부의 상품 추출
            item_list = get_item(target)
            for item in item_list:
                try:
                    itemName = item.img.attrs['alt']
                    itemUrl = item.a.attrs['href']
                except AttributeError as e:
                    print('no item')
                    break
                    
                os.makedirs('downdown/'+itemName)
                get_img(mainUrl+itemUrl,itemName)
                text = get_information(mainUrl+itemUrl)
                get_infofile(text,itemName)
            
            
            if eq(get_nextpage(target),'#none'):
                break
            else:
                target = target+get_nextpage(target)
            count += 1

In [11]:
crawling()

BEST 50  :  /product/list.html?cate_no=76
ALL  :  /product/list.html?cate_no=70
OUTER  :  /product/list.html?cate_no=24
TOP  :  /product/list.html?cate_no=26
SHIRT  :  /product/list.html?cate_no=27
PANTS  :  /product/list.html?cate_no=28
BAG/SHOES  :  /product/list.html?cate_no=42
ACC  :  /product/list.html?cate_no=43
PERSONAL ORDER  :  /product/list.html?cate_no=44
start BEST 50 crawling
1  page
1 ) name :  모던스윗 선글라스 4color
url : /product/detail.html?product_no=1149&cate_no=76&display_group=1
2 ) name :  데일리 여름 가디건
url : /product/detail.html?product_no=528&cate_no=76&display_group=1
3 ) name :  샤인 린넨 밴딩 슬랙스 10color
url : /product/detail.html?product_no=1076&cate_no=76&display_group=1
4 ) name :  베이직 반팔 티셔츠 20color
url : /product/detail.html?product_no=995&cate_no=76&display_group=1
5 ) name :  데니브 슬라브 긴팔 티 20color
url : /product/detail.html?product_no=1130&cate_no=76&display_group=1
6 ) name :  코튼 20수 워싱 반팔 티 10color
url : /product/detail.html?product_no=1021&cate_no=76&display_group=

http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EC%B9%B4%ED%82%A4%29%20%2815%29_shop1_203119.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EC%B9%B4%ED%82%A4%29%20%2816%29_shop1_203119.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EC%B9%B4%ED%82%A4%29%20%2817%29_shop1_203119.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/IMG_4229_shop1_203129.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EC%B9%B4%ED%82%A4%29%20%2812%29_shop1_205613.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EC%B9%B4%ED%82%A4%29%20%2813%29_shop1_205613.jpg
http://mod

http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EB%B8%94%EB%9E%99%29%20%287%29_shop1_210848.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EB%B8%94%EB%9E%99%29%20%288%29_shop1_210848.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EB%B8%94%EB%9E%99%29%20%289%29_shop1_210848.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EB%B8%94%EB%9E%99%29%20%2810%29_shop1_210848.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20%EC%84%A0%EA%B8%80%EB%9D%BC%EC%8A%A4%28%EB%B8%94%EB%9E%99%29%20%2811%29_shop1_210848.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190709/%EB%AA%A8%EB%8D%98%EC%8A%A4%EC%9C%97%20

http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC%8B%9C%EC%8A%A4%EB%A3%A8%EA%B0%80%EB%94%94%EA%B1%B4%20%282%29_shop1_173437.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC%8B%9C%EC%8A%A4%EB%A3%A8%EA%B0%80%EB%94%94%EA%B1%B4%20%2810%29_shop1_173437.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC%8B%9C%EC%8A%A4%EB%A3%A8%EA%B0%80%EB%94%94%EA%B1%B4%20%283%29_shop1_173440.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC%8B%9C%EC%8A%A4%EB%A3%A8%EA%B0%80%EB%94%94%EA%B1%B4%20%284%29_shop1_173440.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC%8B%9C%EC%8A%A4%EB%A3%A8%EA%B0%80%EB%94%94%EA%B1%B4%20%285%29_shop1_173440.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC%8B%9C%EC%8A%A4%EB%A3%A8%EA%B0%80%EB%94%94%EA%B1%B4%20%286%29_shop1_173440.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20180527/%EC%97%AC%EB%A6%84%EC

http://modernsweet.co.kr/web/upload/NNEditor/20190530/NEW%20%EB%A6%B0%EB%84%A8%20%EB%B0%B4%EB%94%A9%20%EC%8A%AC%EB%9E%99%EC%8A%A4%2010color%20%2857%29_shop1_150841.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190530/NEW%20%EB%A6%B0%EB%84%A8%20%EB%B0%B4%EB%94%A9%20%EC%8A%AC%EB%9E%99%EC%8A%A4%2010color%20%2858%29_shop1_150841.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190530/NEW%20%EB%A6%B0%EB%84%A8%20%EB%B0%B4%EB%94%A9%20%EC%8A%AC%EB%9E%99%EC%8A%A4%2010color%20%2859%29_shop1_150841.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190530/NEW%20%EB%A6%B0%EB%84%A8%20%EB%B0%B4%EB%94%A9%20%EC%8A%AC%EB%9E%99%EC%8A%A4%2010color%20%2860%29_shop1_150841.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190530/NEW%20%EB%A6%B0%EB%84%A8%20%EB%B0%B4%EB%94%A9%20%EC%8A%AC%EB%9E%99%EC%8A%A4%2010color%20%2861%29_shop1_150841.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190530/NEW%20%EB%A6%B0%EB%84%A8%20%EB%B0%B4%EB%94%A9%20%EC%8A%AC%EB%9E%99%EC%8A%A4%2010color%20%2862%29_shop1_150

KeyboardInterrupt: 